<a href="https://colab.research.google.com/github/PaoloBarba/ADM_HW2-Group18/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algorithmic Methods of Data Mining**

#Academic year 2022–2023

# Homework 2 - Instagram Profiles & Posts

Authors: Barba Paolo, Bellaroba Albachiara, Soukaina Alaoui.

#Import the packages and upload datasets


In [2]:
import pandas as pd
import random
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm     


In [3]:
from google.colab import drive 
drive.mount('/content/drive/')

MessageError: ignored

In [ ]:
locations =pd.read_csv('/content/drive/MyDrive/ADM_HW2/instagram_locations.csv',sep='\t')
profiles =pd.read_csv("/content/drive/MyDrive/ADM_HW2/instagram_profiles.csv",sep='\t')
posts =pd.read_csv("/content/drive/MyDrive/ADM_HW2/instagram_posts.csv",sep='\t',nrows=1000000)

# RQ1 
## Exploratory Data Analysis

To perform an explanatory data analysis, we will go through three different steps: data explorations, data manipulations, and data summary.
First, we will show the first few rows of the datasets we are analyzing.

In [ ]:
locations.head()

In [ ]:
profiles.head()

In [ ]:
posts.head()

We can now look at the columns of our datasets to understand which variables we are analyzing.
First of all, we will do info method into the dataset to quickly understand the variables we are working with, get the number of rows and
columns we have for each dataset.

In [ ]:
locations.info()

In [ ]:
profiles.info()

In [ ]:
posts.info()

In [ ]:
print("Locations dataframe has " + str(locations.shape[0]) + " observations and "  + str(locations.shape[1]) + " variables")
print("Profiles dataframe has " + str(profiles.shape[0]) + " observations and "  + str(profiles.shape[1]) + " variables")
print("Posts dataframe has " + str(posts.shape[0]) + " observations and "  + str(posts.shape[1]) + " variables")

We are going one step further, having a better understanding of the statistical properties of these data frames by using a described method that gives us basic statistics of all the numerical columns we have. We are doing a statistical summarization at all the numeric variables that have the sense to summarize.

In [ ]:
profiles[["following" , "followers" , "n_posts" ]].describe()

In [ ]:
posts[["numbr_likes","number_comments"]].describe()

To do a better analysis, we clean up our datasets as much as possible by following the steps described below.


1.   **Dealing with duplicates rows**:
      we drop every duplicates rows because contains useless informations 
2.    **Dealing with missing values**:
      we check the percentage of missing values in each column, we drop a column that contains missing values over a set threshold. After we select some columns of interest and drop the NA
3. **Dealing with outliers:** It is risky to include outliers in data-driven models. For some variables of interest, we have incorrect typing data or false profiles that influence the distribution so much, we want to detect them and decide whether to delete them or not.





In [ ]:
#Duplicates rows
profiles.drop_duplicates(inplace=True)
posts.drop_duplicates(inplace=True)
locations.drop_duplicates(inplace=True)

In [ ]:
#Dealing with missing values, columns that contains so much Na
#check for null values / % of null values we have
print('Null values percentage for locations dataset')
print(locations.isnull().sum()/locations.shape[0])
print('\n')
print('Null values percentage for profiles dataset')
print(profiles.isnull().sum()/profiles.shape[0])
print('\n')
print('Null values percentage for posts dataset')
print(posts.isnull().sum()/posts.shape[0])

Null values percentage for locations dataset
sid                       0.000000
id                        0.000000
name                      0.000000
street                    0.300153
zip                       0.300275
city                      0.083598
region                    0.998279
cd                        0.081795
phone                     0.411643
aj_exact_city_match       0.021657
aj_exact_country_match    0.021657
blurb                     0.602306
dir_city_id               0.515285
dir_city_name             0.515285
dir_city_slug             0.515751
dir_country_id            0.515353
dir_country_name          0.515285
lat                       0.006026
lng                       0.006026
primary_alias_on_fb       0.583897
slug                      0.079196
website                   0.390547
cts                       0.000000
dtype: float64


Null values percentage for profiles dataset
sid                    0.000000
profile_id             0.007195
profile_name           0.

In [ ]:
#remove null colums over the threshold
profiles.dropna(thresh = len(profiles)*.6, axis=1)
locations.dropna(thresh = len(locations)*.6, axis=1)

We have to manage Na and outliers values to do a better analysis.
For Na values, we have more than one option to do.

**Replace with the mean:** if there are a lot of missing values we are concentrating the distribution in the mean value and we lose information about the true shape of the distributions.

**Drop Na:** We are losing a relevant percentage of the information we have, however having a huge quantity of data, doesn't influence so much the distribution we are analyzing. So in this case we will go through this procedure .

In [ ]:
#We show only some example of variable we are cleaning
posts.dropna(subset=['numbr_likes', 'number_comments'])

The existence of misleading values has the potential to change the conclusion implied by the model. It is, therefore, important to detect and then decide whether to remove it or not from the dataset. Sometimes the data point may be extremely high or low but that does not mean it is an outlier that we want to get rid of. In the case of Instagram, for instance, a huge amount of followers' value is no sign of errors but can correspond to influencer accounts. On the opposite, an account with zero posts or zero followers usually corresponds to a fake or new account that we don't want to consider in our analysis (we will clean them up when we will analyze them in the next questions) .



In [ ]:
num_bins = 20
plt.figure(figsize=(12,6))

# Logaritmic histogram of the data and box plot
plt.subplot(121)
plt.hist(posts['numbr_likes'] ,  num_bins, facecolor='lightblue', alpha=0.5,log=True)
plt.xlabel("log-value of number of likes")
plt.ylabel("density")
plt.title("Log distribution of \n number of likes per account", size = 16)

plt.subplot(122)
plt.hist(posts['number_comments'] ,  num_bins, facecolor='green', alpha=0.5,log=True)
plt.xlabel("log-value of number of comments")
plt.ylabel("density")
plt.title("Log distribution of \n number of comments per account", size = 16)

plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
plt.boxplot(posts['numbr_likes'] )
plt.xlabel("")
plt.ylabel("number of likes")
plt.title("Box-Plot of number of likes", size = 16)

plt.subplot(122)
plt.boxplot(posts['number_comments'])
plt.xlabel("")
plt.ylabel("number of comments")
plt.title("Box-Plot of number of comments", size = 16)

plt.show()

We used a logarithmic scale because the data covers a wide range of values and there is an evident skewness toward small values.

Since most of the observations are concentrating to low value, we can show the actual distribution untill the 90% percentile.

In [ ]:
likes_outliers= posts[(posts.numbr_likes) < profiles.numbr_likes.quantile(0.95)]
comments_outliers= posts[(profiles.numbr_likes) < profiles.numbr_likes.quantile(0.95)]

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
sns.distplot(likes_outliers, hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4}).set(title='Distribution of \n number of likes per post')

sns.distplot(comments_outliers, hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4}).set(title='Distribution of \n number of comments per post')

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
plt.boxplot(likes_outliers )
plt.xlabel("")
plt.ylabel("number of likes")
plt.title("Box-Plot of number of likes", size = 16)

plt.subplot(122)
plt.boxplot(comments_outlier)
plt.xlabel("")
plt.ylabel("number of comments")
plt.title("Box-Plot of number of comments", size = 16)

plt.show()

# RQ7

In [ ]:
n = 42710197
s = 1000
filename = "/content/drive/MyDrive/ADM_HW2/instagram_posts.csv"
df2= profiles[['followers', 'sid']]
l=[]
for i in range(1000):
  skip = sorted(random.sample(range(n),n-s))
  df = pd.read_csv(filename, usecols= ['numbr_likes' , 'sid_profile'], skiprows=skip, sep='\t')
  df1=posts[['numbr_likes' , 'sid_profile']]
  df1=df1.dropna()
  df2=df2.dropna()
  data=pd.merge(df1, df2, left_on="sid_profile", right_on="sid").drop('sid', axis=1)
  l.append(sum(data.apply(lambda x :  x.numbr_likes / x.followers, axis = 1 ) > 0.20)/ data.shape[0])




ValueError: ignored

In [ ]:
pd.read_csv(filename, sep='\t', usecols= ['numbr_likes' , 'sid_profile'],)